In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from ConvBN1d import ConvBN
from LinearBN import LinearBN
from source import train, test

In [2]:
def load_ucr(file):
    data = np.loadtxt(file)
    X = data[:, 1:]
    y = data[:, 0]
    y = np.where(y == 1, 1, 0)  # convert labels to 0/1
    return X, y

# Adjust file paths to your local files
X_train, y_train = load_ucr("../FordA_TRAIN.txt")
X_test, y_test = load_ucr("../FordA_TEST.txt")

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (3601, 500), Test shape: (1320, 500)


In [3]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [4]:
mean = X_train_tensor.mean()
std = X_train_tensor.std()
X_train_tensor = (X_train_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
class TanH(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Parameter(torch.tensor(10.0)) 
    def forward(self, x):
        return torch.nn.functional.tanh(self.alpha*x)

In [10]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1_out = 16
        self.conv1_size = 15
        self.conv1_padding = 7


        self.conv2_out = 16
        self.conv2_size = 15
        self.conv2_padding = 7

        self.conv3_out = 25
        self.conv3_size = 13
        self.conv3_padding = 6

        self.fc1_out = 2

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN(in_channels=1, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .05, bias_par_init=0.0015)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .05, bias_par_init=0.0015)
        self.block3 = ConvBN(in_channels=self.conv2_out, out_channels=self.conv3_out, kernel_size=self.conv3_size, padding=self.conv3_padding, std = .05, bias_par_init=0.0015)
               
        
        # torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.conv3_out * (500//2//2//2), self.fc1_out))
        nn.init.normal_(self.w2, mean=0.0, std=.05)

        self.dropout = nn.Dropout(0.5)

        self.tanh = TanH()




    def forward(self, x):
        x = F.max_pool1d(self.tanh(self.block1(x)), 2)
        x = F.max_pool1d(self.tanh(self.block2(x)), 2)
        x = F.max_pool1d(self.tanh(self.block3(x)), 2)
        
        x = x.view(x.size(0), -1)
        
        # x = self.relu(self.block3(x))
        # x = self.dropout(x)

        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

In [11]:
import torch.optim as optim
import time  # Import time module

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training = True
model = Network().to(device)
criterion = nn.CrossEntropyLoss()
# Trained 100 Epochs with lr=0.025, 50 epochs with 0.005 and 50 epochs with 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

if training:
    model.train()
    loss_hist, acc_hist = [], []    
    
    for epoch in range(20):
        start_time = time.time()  # Record the start time of the epoch
    
        running_loss = 0.0
        correct = 0
        for data in train_loader:
            batch, labels = data
            batch, labels = batch.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # Compute training statistics
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
    
        avg_loss = running_loss / len(train_dataset)
        avg_acc = correct / len(train_dataset)
        loss_hist.append(avg_loss)
        acc_hist.append(avg_acc)
        
    
        # Calculate elapsed time
        elapsed_time = time.time() - start_time  # Calculate the time taken for this epoch
    
        print('[epoch %d] loss: %.5f accuracy: %.4f time: %.2f seconds' %
              (epoch + 1, avg_loss, avg_acc, elapsed_time))


[epoch 1] loss: 0.01711 accuracy: 0.6995 time: 0.77 seconds
[epoch 2] loss: 0.00700 accuracy: 0.9056 time: 0.77 seconds
[epoch 3] loss: 0.00603 accuracy: 0.9247 time: 0.77 seconds
[epoch 4] loss: 0.00530 accuracy: 0.9306 time: 0.77 seconds
[epoch 5] loss: 0.00458 accuracy: 0.9475 time: 0.77 seconds
[epoch 6] loss: 0.00427 accuracy: 0.9478 time: 0.77 seconds
[epoch 7] loss: 0.00422 accuracy: 0.9497 time: 0.77 seconds
[epoch 8] loss: 0.00364 accuracy: 0.9595 time: 0.77 seconds
[epoch 9] loss: 0.00314 accuracy: 0.9664 time: 0.77 seconds
[epoch 10] loss: 0.00309 accuracy: 0.9656 time: 0.77 seconds
[epoch 11] loss: 0.00292 accuracy: 0.9686 time: 0.77 seconds
[epoch 12] loss: 0.00269 accuracy: 0.9722 time: 0.77 seconds
[epoch 13] loss: 0.00259 accuracy: 0.9745 time: 0.77 seconds
[epoch 14] loss: 0.00236 accuracy: 0.9770 time: 0.79 seconds
[epoch 15] loss: 0.00218 accuracy: 0.9817 time: 0.77 seconds
[epoch 16] loss: 0.00207 accuracy: 0.9819 time: 0.78 seconds
[epoch 17] loss: 0.00195 accuracy

In [12]:
model.eval()  # Set the model to evaluation mode

test_loss = 0.0
correct_test = 0

# Evaluate on the test dataset
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Get predictions and update the correct count
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()

# Compute average loss and accuracy for the test set
avg_test_loss = test_loss / len(test_dataset)
avg_test_acc = correct_test / len(test_dataset)

print(f"Test loss: {avg_test_loss:.5f}, Test accuracy: {avg_test_acc:.4f}")

Test loss: 0.00580, Test accuracy: 0.9288


In [13]:
torch.save(model.state_dict(), 'FordA_GNet_Training.pth')